# Using Examon with Apache Spark
This example shows how to set the examon-client to execute distributed queries and analytics with Apache Spark.
The example covers both the JSON and the ExamonQL query interfaces.

In [1]:

import os
from examon.examon import Examon, ExamonQL
from examon.executors import ExSpark

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession

#### Configure Spark

In [3]:

# set the spark configuration
conf = SparkConf()
conf.setAppName('ExamonClientTestApp')
conf.setMaster('local[*]')

# create the sparkcontext and sparksession objects 
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)  # needed for toDF

#### Configure the Examon client

In [4]:

KAIROSDB_SERVER = '130.186.13.80'
KAIROSDB_PORT = '3000'
USER = ''
PWD = ''
ex = Examon(KAIROSDB_SERVER, port=KAIROSDB_PORT, user=USER, password=PWD, verbose=False, proxy=True)


### 1) Query in Spark using the native db interface (JSON)

In [5]:
# query some data
tstart = "12-06-2019 08:09:00"   
tstop = "19-06-2019 19:09:00"
metrics = ['Sys_Power']
tags = {'node': ['r162c16s02']}
groupby = {'name':'tag', 'tags':['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}
aggrby = None

# create the spark executor, initialized with the examon-client data
sp = ExSpark(ex)

# define a Spark RDD with the content of the query
rdd = sp \
    .spark(sc) \
    .query(tstart, tstop, metrics, tags, groupby=groupby, aggrby=aggrby, time_zone='Europe/Rome', batch_size=30*60*1000) \
    .to_rdd(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': ['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}], aggrby=None, limit=None, time_zone='Europe/Rome')


Please note: The original query is sliced in chunks over the total time range. Their size is defined by the *batch_size* parameter (milliseconds) of the *query()* method. So each Spark worker will query an amount of data defined by the number of chunks per partition. The number of partitions of the resulting RDD can be defined by the *npartitions* parameter of the *to_rdd()* method. If you are not sure, you can leave *npartitions=None* (default) and it will be handled by Spark as usual.

In [6]:
# Trigger the query executing an action (take) on the RDD
rdd.take(10)

[Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319740028, type=u'Current', units=u'W', value=170),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319800032, type=u'Current', units=u'W', value=110),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319860028, type=u'Current', units=u'W', value=340),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319920034, type=u'Current', units=u'W', value=340),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'conflue

In [7]:
# cretae a Spark dataframe from the RDD
df = rdd.toDF()
df.show(10)

+----+-------+------+---------+----------+------+-------+-------------+-------------+-------+-----+-----+
|chnl|cluster|health|     name|      node|   org|   part|       plugin|    timestamp|   type|units|value|
+----+-------+------+---------+----------+------+-------+-------------+-------------+-------+-----+-----+
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319740028|Current|    W|  170|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319800032|Current|    W|  110|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319860028|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319920034|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319980026|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560320040030|Current|    W|  350|
|data|marconi|    ok|Sys_Power|r162c16s02|cine

In [8]:
# show the inferred column data types of the spark dataframe
df.dtypes

[('chnl', 'string'),
 ('cluster', 'string'),
 ('health', 'string'),
 ('name', 'string'),
 ('node', 'string'),
 ('org', 'string'),
 ('part', 'string'),
 ('plugin', 'string'),
 ('timestamp', 'bigint'),
 ('type', 'string'),
 ('units', 'string'),
 ('value', 'bigint')]

### 2) Query in Spark using the ExamonQL interface
Here we do the same query but showing how the ExamonQL interface can be used with the Spark executor

In [10]:
# create the ExamonQl object
sq = ExamonQL(ex)

# do the query as usual but get only the query data using 'get_query()'
query = sq.SELECT('*') \
    .FROM('Sys_Power') \
    .WHERE(node='r162c16s02') \
    .TSTART('12-06-2019 08:09:00') \
    .TSTOP('19-06-2019 19:09:00') \
    .get_query()

In [11]:
# pass the query data to the spark executor 
sp = ExSpark(ex)

# define the RDD passing the query data
rdd = sp \
      .spark(sc) \
      .query(*query, batch_size=30*60*1000) \
      .to_rdd(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': [u'node', u'plugin', u'chnl', u'cluster', u'part', u'health', u'units', u'org', u'type']}], aggrby=None, limit=None, time_zone='Europe/Rome')


In [12]:
# Trigger the query executing an action (take) on the RDD
rdd.take(10)

[Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319740028, type=u'Current', units=u'W', value=170),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319800032, type=u'Current', units=u'W', value=110),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319860028, type=u'Current', units=u'W', value=340),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'confluent_pub', timestamp=1560319920034, type=u'Current', units=u'W', value=340),
 Row(chnl=u'data', cluster=u'marconi', health=u'ok', name=u'Sys_Power', node=u'r162c16s02', org=u'cineca', part=u'skylake', plugin=u'conflue

In [13]:
# cretae a Spark dataframe from the RDD
df = rdd.toDF()
df.show(10)

+----+-------+------+---------+----------+------+-------+-------------+-------------+-------+-----+-----+
|chnl|cluster|health|     name|      node|   org|   part|       plugin|    timestamp|   type|units|value|
+----+-------+------+---------+----------+------+-------+-------------+-------------+-------+-----+-----+
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319740028|Current|    W|  170|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319800032|Current|    W|  110|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319860028|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319920034|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560319980026|Current|    W|  340|
|data|marconi|    ok|Sys_Power|r162c16s02|cineca|skylake|confluent_pub|1560320040030|Current|    W|  350|
|data|marconi|    ok|Sys_Power|r162c16s02|cine

In [14]:
# show the inferred column data types of the spark dataframe
df.dtypes

[('chnl', 'string'),
 ('cluster', 'string'),
 ('health', 'string'),
 ('name', 'string'),
 ('node', 'string'),
 ('org', 'string'),
 ('part', 'string'),
 ('plugin', 'string'),
 ('timestamp', 'bigint'),
 ('type', 'string'),
 ('units', 'string'),
 ('value', 'bigint')]